In [1]:
import tensorflow as tf  
import numpy as np       
import pandas as pd       
from matplotlib import pyplot as plt   
import seaborn as sns  
%matplotlib inline    
import time          
from tensorflow.keras.callbacks import History       
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard       
from tensorflow.keras import metrics          
from sklearn.metrics import confusion_matrix          
from keras.callbacks import TensorBoard    
import pickle
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Multiply, Dense, Conv1D
from tensorflow.keras import backend as K
from tensorflow.keras.layers import add
import platform
import os

In [2]:
# Display Settings
pd.options.display.max_columns = None
pd.options.display.max_rows = None
sns.set()

In [3]:
# System Information Print the Python version
print("Python version: ", platform.python_version())
print("TensorFlow version: ", tf.__version__)
print("Current working directory: ", os.getcwd())
!uname -a
!nvidia-smi

Python version:  3.11.7
TensorFlow version:  2.15.0
Current working directory:  /root/.jupyter/张彤/注意力机制
Linux w3q2ulc9.vm 5.15.0-60-generic #66-Ubuntu SMP Fri Jan 20 14:29:49 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux
Sat Oct 26 20:30:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        On  | 00000000:09:00.0 Off |                  Off |
|  0%   24C    P8               8

In [4]:
# Hyperparameter Settings
EPOCHS = 50
IMAGE_SIZE = (224, 224)
IMAGE_PATH = "../data"
LEARNING_RATE = 1e-4
BATCH_SIZE = 64

In [5]:
# Load Dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    IMAGE_PATH,
    validation_split=0.2,
    subset="training",    
    seed=123,               
    image_size=IMAGE_SIZE,   
    batch_size=BATCH_SIZE    
)

vaild_ds = tf.keras.utils.image_dataset_from_directory(
    IMAGE_PATH,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

Found 14080 files belonging to 100 classes.
Using 11264 files for training.


2024-10-26 20:30:10.691769: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-26 20:30:10.692250: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-26 20:30:11.011449: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Found 14080 files belonging to 100 classes.
Using 2816 files for validation.


In [6]:
# Image Normalization
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
mean_tensor = tf.constant(mean, dtype=tf.float32)
std_tensor = tf.constant(std, dtype=tf.float32)

In [7]:
# Data Augmentation
train_image_augment = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1 / 255.0),
    tf.keras.layers.RandomRotation(factor=0.2),
    tf.keras.layers.RandomFlip(),
])

valid_image_augment = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1 / 255.0),
])

In [8]:
# Input Processing
def process_train_input(images, labels):
    return train_image_augment(images), labels

def process_valid_input(images, labels):
    return valid_image_augment(images), labels


In [9]:
# Data Preprocessing
def convert_types_and_encode(x, y):
    y = tf.cast(y, tf.int32)  
    y_one_hot = tf.one_hot(y, 100)  
    return x, y_one_hot

In [10]:
# Apply Data Preprocessing
train_ds = train_ds.map(convert_types_and_encode)
train_ds = train_ds.map(process_train_input, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

vaild_ds = vaild_ds.map(convert_types_and_encode)
vaild_ds = vaild_ds.map(process_valid_input, num_parallel_calls=tf.data.AUTOTUNE)
vaild_ds = vaild_ds.prefetch(tf.data.AUTOTUNE)

In [11]:
# Define CBAM block and MobileNet model
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Multiply, Dense, Conv1D, Add
from tensorflow.keras.applications import MobileNet
from tensorflow.keras import backend as K

def cbam_block(input_tensor, reduction_ratio=16):
    channel = input_tensor.shape[-1]

    # Channel Attention
    avg_pool = GlobalAveragePooling2D()(input_tensor)
    avg_pool = Reshape((1, 1, channel))(avg_pool)
    avg_fc = Dense(channel // reduction_ratio, activation='relu')(avg_pool)
    avg_fc = Dense(channel, activation='sigmoid')(avg_fc)

    max_pool = GlobalMaxPooling2D()(input_tensor)
    max_pool = Reshape((1, 1, channel))(max_pool)
    max_fc = Dense(channel // reduction_ratio, activation='relu')(max_pool)
    max_fc = Dense(channel, activation='sigmoid')(max_fc)

    # Channel Attention: Add the outputs of avg_pool and max_pool, then apply sigmoid activation
    channel_attention = Add()([avg_fc, max_fc])  # Use Add() for element-wise addition
    channel_attention = tf.keras.activations.sigmoid(channel_attention)

    # Refine the input tensor with channel attention
    channel_refined = Multiply()([input_tensor, channel_attention])

    # Spatial Attention
    spatial_avg_pool = K.mean(input_tensor, axis=-1, keepdims=True)
    spatial_max_pool = K.max(input_tensor, axis=-1, keepdims=True)
    spatial_concat = K.concatenate([spatial_avg_pool, spatial_max_pool], axis=-1)
    spatial_conv = Conv1D(1, kernel_size=7, padding='same', activation='sigmoid')(spatial_concat)

    # Refine the input tensor with spatial attention
    spatial_attention = Multiply()([channel_refined, spatial_conv])

    return spatial_attention

# Create MobileNet base model
base_model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Apply CBAM block
feature_maps = base_model.output
cbam_feature_maps = cbam_block(feature_maps)

# Add global average pooling and dense layers
x = GlobalAveragePooling2D()(cbam_feature_maps)
x = Dense(1024, activation='relu')(x)
outputs = Dense(100, activation='softmax')(x)

# Create the final model
model = tf.keras.Model(inputs=base_model.input, outputs=outputs)


In [12]:
# Compile Model
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
loss_fn = tf.keras.losses.CategoricalCrossentropy()
metrics = [
    tf.keras.metrics.CategoricalAccuracy(),
    tf.keras.metrics.Precision(),
    tf.keras.metrics.Recall(),
]

model.compile(optimizer=optimizer, loss=loss_fn, metrics=metrics)

In [14]:
# Train Model
log_dir = "../Running result/CBAM/CBAM"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=2, min_lr=1e-8)

history = model.fit(train_ds, epochs=EPOCHS, validation_data=vaild_ds, callbacks=[lr_scheduler, tensorboard_callback])

Epoch 1/50
176/176 [==============================] - 16s 86ms/step - loss: 0.0202 - categorical_accuracy: 0.9948 - precision: 0.9949 - recall: 0.9944 - val_loss: 0.3245 - val_categorical_accuracy: 0.9119 - val_precision: 0.9271 - val_recall: 0.9031 - lr: 6.2500e-06
Epoch 2/50
176/176 [==============================] - 15s 84ms/step - loss: 0.0206 - categorical_accuracy: 0.9948 - precision: 0.9949 - recall: 0.9945 - val_loss: 0.3248 - val_categorical_accuracy: 0.9119 - val_precision: 0.9271 - val_recall: 0.9031 - lr: 6.2500e-06
Epoch 3/50
176/176 [==============================] - 16s 86ms/step - loss: 0.0206 - categorical_accuracy: 0.9942 - precision: 0.9947 - recall: 0.9936 - val_loss: 0.3286 - val_categorical_accuracy: 0.9098 - val_precision: 0.9256 - val_recall: 0.9016 - lr: 6.2500e-06
Epoch 4/50
176/176 [==============================] - 15s 84ms/step - loss: 0.0187 - categorical_accuracy: 0.9950 - precision: 0.9956 - recall: 0.9949 - val_loss: 0.3262 - val_categorical_accuracy: 0

In [15]:
import pandas as pd  # Import Pandas library
import os

# Define save path
model_dir = "../Running result/CBAM/CBAM"
excel_file_path = os.path.join(model_dir, "CBAM.xlsx")  # Path to save the Excel file
model_file_path = os.path.join(model_dir, "CBAM.h5")  # Path to save the model

# Check if the save directory exists; create it if it does not
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Save training history to an Excel file
history_df = pd.DataFrame(history.history)  
history_df.to_excel(excel_file_path, index=False)
print(f"Training history saved to: {excel_file_path}")

# Save the model to the specified path
model.save(model_file_path)
print(f"Model saved to: {model_file_path}")


Training history saved to: ../Running result/CBAM/CBAM/CBAM.xlsx


/root/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved to: ../Running result/CBAM/CBAM/CBAM.h5
